In [1]:
from ray_trading_env import RayTradingEnvironment

INFO:ray_trading_env:ray_trading_env logger started.


In [2]:
import numpy as np
import pandas as pd
import pprint
import ray
from ray.rllib.agents.ppo import PPOTrainer

In [3]:
# Start a new instance of Ray
# Note: execute cell only once!
# instance can be stopped with ray.shutdown()
ray.init()

2022-06-27 14:25:04,985	WARNING services.py:2002 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


RayContext(dashboard_url='', python_version='3.8.8', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '10.1.150.226', 'raylet_ip_address': '10.1.150.226', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-27_14-25-03_365442_27490/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-27_14-25-03_365442_27490/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-06-27_14-25-03_365442_27490', 'metrics_export_port': 60507, 'gcs_address': '10.1.150.226:58938', 'address': '10.1.150.226:58938', 'node_id': '8c27d2d27bfe09f6c3c9ab027b13f77e6e3d8d39f55f678c75ba0df9'})

In [4]:
# config dict for Rllib
config = {
    "env": RayTradingEnvironment,
    "env_config": {
        "config": {
            "trading_days": 252,
            "trading_cost_bps": 1e-3,
            "time_cost_bps": 1e-4,
            "ticker": "AAPL",
            "get_data_from": "csv", # "csv" or "hdf"
            #"max_episode_steps": 252,
        },
    },

    "create_env_on_driver": True,
    # horizon needs to be specified if the env has no 
    # max_number_of_steps-like parameter
    "horizon" : 252,
}

In [5]:
# Instantiate the Trainer object using above config.
rllib_trainer = PPOTrainer(config=config)

2022-06-27 14:25:06,222	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-06-27 14:25:06,226	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-06-27 14:25:06,226	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
INFO:ray_trading_env:got data for AAPL...
INFO:ray_trading_env:None
/opt/conda/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2022-06-27 14:25:06,307	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have 

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9367 entries, (Timestamp('1981-01-30 00:00:00'), 'AAPL') to (Timestamp('2018-03-27 00:00:00'), 'AAPL')
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   returns  9367 non-null   float64
 1   ret_2    9367 non-null   float64
 2   ret_5    9367 non-null   float64
 3   ret_10   9367 non-null   float64
 4   ret_21   9367 non-null   float64
 5   rsi      9367 non-null   float64
 6   macd     9367 non-null   float64
 7   atr      9367 non-null   float64
 8   stoch    9367 non-null   float64
 9   ultosc   9367 non-null   float64
dtypes: float64(10)
memory usage: 1.1+ MB


2022-06-27 14:25:08,128	WARNING util.py:65 -- Install gputil for GPU system monitoring.


In [6]:
# check the configuration of the trainer instance
rllib_trainer.get_config()

{'num_workers': 2,
 'num_envs_per_worker': 1,
 'create_env_on_driver': True,
 'rollout_fragment_length': 200,
 'batch_mode': 'truncate_episodes',
 'gamma': 0.99,
 'lr': 5e-05,
 'train_batch_size': 4000,
 'model': {'_use_default_native_models': False,
  '_disable_preprocessor_api': False,
  '_disable_action_flattening': False,
  'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'post_fcnet_hiddens': [],
  'post_fcnet_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': False,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'use_attention': False,
  'attention_num_transformer_units': 1,
  'attention_dim': 64,
  'attention_num_heads': 1,
  'attention_head_dim': 32,
  'attention_memory_inference': 50,
  'attention_memory_training': 50,
  'attention_position_wise_mlp_dim': 32,


In [ ]:
# train the trainer instance
for _ in range(10):
    results = rllib_trainer.train()
    print(f"Iteration={rllib_trainer.iteration}: R(\"return\")={results['episode_reward_mean']}")

(RolloutWorker pid=28419) INFO:ray_trading_env:ray_trading_env logger started.
(RolloutWorker pid=28418) INFO:ray_trading_env:ray_trading_env logger started.
(RolloutWorker pid=28418) INFO:ray_trading_env:got data for AAPL...
(RolloutWorker pid=28418) INFO:ray_trading_env:None
(RolloutWorker pid=28418) /opt/conda/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=28418)   logger.warn(
(RolloutWorker pid=28418) 2022-06-27 14:25:08,748	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


(RolloutWorker pid=28418) <class 'pandas.core.frame.DataFrame'>
(RolloutWorker pid=28418) MultiIndex: 9367 entries, (Timestamp('1981-01-30 00:00:00'), 'AAPL') to (Timestamp('2018-03-27 00:00:00'), 'AAPL')
(RolloutWorker pid=28418) Data columns (total 10 columns):
(RolloutWorker pid=28418)  #   Column   Non-Null Count  Dtype  
(RolloutWorker pid=28418) ---  ------   --------------  -----  
(RolloutWorker pid=28418)  0   returns  9367 non-null   float64
(RolloutWorker pid=28418)  1   ret_2    9367 non-null   float64
(RolloutWorker pid=28418)  2   ret_5    9367 non-null   float64
(RolloutWorker pid=28418)  3   ret_10   9367 non-null   float64
(RolloutWorker pid=28418)  4   ret_21   9367 non-null   float64
(RolloutWorker pid=28418)  5   rsi      9367 non-null   float64
(RolloutWorker pid=28418)  6   macd     9367 non-null   float64
(RolloutWorker pid=28418)  7   atr      9367 non-null   float64
(RolloutWorker pid=28418)  8   stoch    9367 non-null   float64
(RolloutWorker pid=28418)  9   u

(RolloutWorker pid=28419) INFO:ray_trading_env:got data for AAPL...
(RolloutWorker pid=28419) INFO:ray_trading_env:None
(RolloutWorker pid=28419) /opt/conda/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=28419)   logger.warn(
(RolloutWorker pid=28419) 2022-06-27 14:25:08,795	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-06-27 14:25:10,690	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


Iteration=1: R("return")=-0.25545044336567463
Iteration=2: R("return")=-0.21998192067462713
Iteration=3: R("return")=-0.2488374324417533
Iteration=4: R("return")=-0.27422901396665567
Iteration=5: R("return")=-0.2565586904981205
